In [6]:
import cohere
import os
import uuid
from typing import List, Dict

COHERE_API_KEY = os.getenv("COHERE_API_KEY")
co = cohere.Client(COHERE_API_KEY)

### Chatbot component

In [12]:
class Chatbot:
    def __init__(self, connectors: List[str]):
        self.conversation_id = str(uuid.uuid4())
        self.connectors = [{"id": c} for c in connectors]
        

    def generate_response(self, message: str):
        response = co.chat(
                        message=message,
                        connectors=self.connectors,
                        conversation_id=self.conversation_id,
                        stream=True
                        )

        for event in response:
                yield event

### App component

In [13]:
class App:
    def __init__(self, chatbot: Chatbot):
        self.chatbot = chatbot
        
    def run(self):
        while True:
            # Get the user message
            message = input("User: ")

            # Typing "quit" ends the conversation
            if message.lower() == "quit":
                print("Ending chat.")
                break
            else:
                print(f"User: {message}")

            # Get the chatbot response
            response = self.chatbot.generate_response(message)

            # Print the chatbot response
            print("Chatbot:")
            
            documents = []
            documents_flag = False
            citations_flag = False
            
            for event in response:
                # Documents
                if event.event_type == "search-results":
                    documents_flag = True
                    documents = event.documents
                    
                # Text
                if event.event_type == "text-generation":
                    print(event.text, end="")        

                # Citations
                if event.event_type == "citation-generation":
                    if not citations_flag:
                        print("\n\nCITATIONS:")
                        citations_flag = True
                    print(event.citations)
            
            if documents_flag:
                print("\n\nDOCUMENTS:")
                for d in documents:
                    print(f'{d["title"]} ({d["id"]}). URL: {d["url"]}')

            print(f"\n{'-'*100}\n")

### Run chatbot with Google Drive connector

In [19]:
# Define connectors
connectors = ["demo-conn-gdrive-1x2p4k"]

# Create an instance of the Chatbot class by supplying the connectors
chatbot = Chatbot(connectors)

# Create an instance of the App class with the Chatbot instance
app = App(chatbot)

# Run the chatbot
app.run()

User: What is prompt engineering
Chatbot:
Prompt engineering is about sending instructions to text generation models and obtaining responses. A prompt can consist of a single line of instruction, but the more specific it is, the more accurate you can expect the response to be. Each additional building block added to a prompt provides a different means of improving the quality of the response.

When working with large language models (LLMs), prompt engineering opens up many possibilities for creativity. However, there are trade-offs in terms of performance, and outputs from LLMs are probabilistic, so a mechanism for validating outputs is necessary.

CITATIONS:
[{'start': 28, 'end': 99, 'text': 'sending instructions to text generation models and obtaining responses.', 'document_ids': ['demo-conn-gdrive-1x2p4k_0:3']}]
[{'start': 126, 'end': 152, 'text': 'single line of instruction', 'document_ids': ['demo-conn-gdrive-1x2p4k_0:3', 'demo-conn-gdrive-1x2p4k_0:33']}]
[{'start': 162, 'end': 23

### Run chatbot with web search connector

In [25]:
# Define connectors
connectors = ["web-search"]

# Create an instance of the Chatbot class by supplying the connectors
chatbot = Chatbot(connectors)

# Create an instance of the App class with the Chatbot instance
app = App(chatbot)

# Run the chatbot
app.run()

User: What is LLM University
Chatbot:
LLM University (LLMU) is an online learning resource provided by Cohere that offers a comprehensive curriculum on natural language processing (NLP) using large language models. It caters to learners from all backgrounds, from beginners to advanced, and the courses are geared towards anyone excited about language processing, including those looking to build apps using language AI. The courses cover everything from the basics of LLMs to advanced topics like generative AI, with practical code examples to help learners solidify their knowledge. The hands-on exercises allow learners to build and deploy their own models.

CITATIONS:
[{'start': 15, 'end': 21, 'text': '(LLMU)', 'document_ids': ['web-search_1:0', 'web-search_0:4', 'web-search_1:2', 'web-search_1:3', 'web-search_0:3']}]
[{'start': 28, 'end': 71, 'text': 'online learning resource provided by Cohere', 'document_ids': ['web-search_1:0']}]
[{'start': 86, 'end': 147, 'text': 'comprehensive curric

### Run chatbot with multiple connectors

In [30]:
# Define connectors
connectors = ["demo-conn-gdrive-1x2p4k", "web-search"]

# Create an instance of the Chatbot class by supplying the connectors
chatbot = Chatbot(connectors)

# Create an instance of the App class with the Chatbot instance
app = App(chatbot)

# Run the chatbot
app.run()

User: What is chain of thought prompting
Chatbot:
Chain of thought prompting is a technique used to help LLMs (Large Language Models) perform complex reasoning by breaking down problems into logical, bite-sized chunks. This method encourages LLMs to produce intermediate reasoning steps before delivering a final answer to a multi-step problem. The idea is that a model-generated chain of thought would mimic an intuitive thought process when working through a multi-step reasoning problem. 

This concept was introduced by Wei et al. in 2023, and has been found to be particularly useful in improving LLMs' performance at complex arithmetic, commonsense, and symbolic reasoning tasks.

CITATIONS:
[{'start': 55, 'end': 83, 'text': 'LLMs (Large Language Models)', 'document_ids': ['web-search_9:2']}]
[{'start': 92, 'end': 109, 'text': 'complex reasoning', 'document_ids': ['web-search_7:2', 'web-search_9:2', 'web-search_8:7', 'web-search_8:1']}]
[{'start': 113, 'end': 168, 'text': 'breaking down p